<a href="https://colab.research.google.com/github/sheffieldcl/NIHL-Training/blob/Python-Training/Week4_sz_Biopython_Introduction_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The **Biopython project** is an open-source collection of non-commercial Python tools for computational biology and bioinformatics, created by an international association of developers.
<p>
It contains classes to represent biological sequences and sequence annotations, and it is able to read and write to a variety of file formats. It also allows for a programmatic means of accessing online databases of biological information, such as those at NCBI. Separate modules extend Biopython's capabilities to sequence alignment, protein structure, population genetics, phylogenetics, sequence motifs, and machine learning.
<p>
Reference <p>
https://en.wikipedia.org/wiki/Biopython

**Biopython** is an amazing python library for bioinformatics, which provides a wide range of functions from reading large files with biological data to aligning sequences

In [ ]:
# install Biopython

! pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.7 MB/s eta 0:00:00


In [ ]:
# import the Biopython package
import Bio

Sequence object creation, manipulation, etc.

In [ ]:
from Bio.Seq import Seq

# create a sequence object
seq = Seq("ATGACGTTGCATG")

print(type(seq))
print(f"seq length {len(seq)}")
print("complement seq: ", seq.complement())
print("reversed complement seq (align from 5' to 3'): ", seq.reverse_complement())
print("The number of adenine (A) in the sequence: ", seq.count("A"))
print("The index of the TTG triplet in the sequence: ", seq.find("TTG"))

<class 'Bio.Seq.Seq'>
seq length 13
complement seq:  TACTGCAACGTAC
reversed complement seq (align from 5' to 3'):  CATGCAACGTCAT
The number of adenine (A) in the sequence:  3
The index of the TTG triplet in the sequence:  6


Use Biopython to fetch GenBank file and FASTA file from NCBI

In [ ]:
# the package name to import is Bio
from Bio import Entrez

#Provide an email address
Entrez.email = "youremail@gmail.com"

#First, you may want to find out the available search fields in the
#database you're searching (the nucleotide database contains DNA/RNA #info, so that's what we want here).
with Entrez.einfo(db="nucleotide") as handle:
    record = Entrez.read(handle)

for field in record["DbInfo"]["FieldList"]:
    print("Name: {}, Full Name: {}, Description: {}".format(field["Name"], field["FullName"], field["Description"]))

###Name: ACCN, Full Name: Accession, Description: Accession number of sequence
###Returns a printout of search fields like this:
###ACCN, Accession, Accession number of sequence
###PACC, Primary Accession, Does not include retired secondary accessions
###GENE, Gene Name, Name of gene associated with sequence
###PROT, Protein Name, Name of protein associated with sequence
###ECNO, EC/RN Number, EC number for enzyme or CAS registry number
###PDAT, Publication Date, Date sequence added to GenBank
###

Name: ALL, Full Name: All Fields, Description: All terms from all searchable fields
Name: UID, Full Name: UID, Description: Unique number assigned to each sequence
Name: FILT, Full Name: Filter, Description: Limits the records
Name: WORD, Full Name: Text Word, Description: Free text associated with record
Name: TITL, Full Name: Title, Description: Words in definition line
Name: KYWD, Full Name: Keyword, Description: Nonstandardized terms provided by submitter
Name: AUTH, Full Name: Author, Description: Author(s) of publication
Name: JOUR, Full Name: Journal, Description: Journal abbreviation of publication
Name: VOL, Full Name: Volume, Description: Volume number of publication
Name: ISS, Full Name: Issue, Description: Issue number of publication
Name: PAGE, Full Name: Page Number, Description: Page number(s) of publication
Name: ORGN, Full Name: Organism, Description: Scientific and common names of organism, and all higher levels of taxonomy
Name: ACCN, Full Name: Accession, Descriptio

The main purpose of the following cell is to get the record ids <p>
We can use Entrez with the retrieved ID to get the FASTA file or GenBank File from the NCBI server

In [ ]:
# Second, let's get all of the accession numbers for the available
# SARS-CoV-2 sequences using the ORGN field (organism) to limit the #search term
# for demo purpose, we just fetch the first 20 records

with Entrez.esearch(db="nucleotide", term="SARS-CoV-2[ORGN]", idtype="acc", retmax="20") as handle:
    results = Entrez.read(handle)

print(results.keys())
#save the list of accession numbers
accs = results["IdList"]
print(accs)

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])
['PP757787.1', 'PP757786.1', 'PP756578.1', 'PP756577.1', 'PP756576.1', 'PP756575.1', 'PP756574.1', 'PP756573.1', 'PP756572.1', 'PP756571.1', 'PP756570.1', 'PP756569.1', 'PP756568.1', 'PP756567.1', 'PP756566.1', 'PP756565.1', 'PP756564.1', 'PP756563.1', 'PP756562.1', 'PP756561.1']


Now we have a list of accession numbers containing sequences from SARS-CoV-2. Next, we want to fetch those records from GenBank using Entrez.efetch() method. The GenBank (.gbk) file format is rather large file (>300 Mb) — if you wanted it to be smaller, you could set rettype="fasta" which returns less information per record.

In [ ]:
import os


if not os.path.exists("./data/raw"):
    os.makedirs("./data/raw")

# Use efetch to download records and save to disk - get the GenBank format at first
filename = "./data/raw/SARS-CoV-2.gbk"


if not os.path.isfile(filename):
    # Downloading...
    # set rettype='gb' then you get the GenBank format file, set rettype='fasta', you get the FAST-A format file
    with Entrez.efetch(db="nucleotide", id=accs, rettype="gb", retmode="text") as net_handle:
        with open(filename, "w") as out_handle:
            out_handle.write(net_handle.read())
    print("Saved")

Saved


Try to download the fasta format file

In [ ]:
filename = "./data/raw/SARS-CoV-2.fasta"

if not os.path.isfile(filename):
    # Downloading...
    with Entrez.efetch(db="nucleotide", id=accs, rettype="fasta", retmode="text") as net_handle:
        with open(filename, "w") as out_handle:
            out_handle.write(net_handle.read())
    print("Saved")

Reading a sequence file <p>
You will find reading a FASTA file is much quicker than the GenBank file <p>
We can read and parse the .gbk file and the .fasta file by SeqIO.parse

In [ ]:
from Bio import SeqIO

for idx, record in enumerate(SeqIO.parse("/content/data/raw/SARS-CoV-2.gbk", "gb")):
    # just print the first record
    if idx < 1:
        print(record)

ID: PP757787.1
Name: PP757787
Description: Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NY-PBRI-64844/2024, complete genome
Number of features: 55
/molecule_type=RNA
/topology=linear
/data_file_division=VRL
/date=03-MAY-2024
/accessions=['PP757787']
/sequence_version=1
/keywords=['']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus', 'Severe acute respiratory syndrome-related coronavirus']
/references=[Reference(title='Direct Submission', ...)]
/structured_comment=defaultdict(<class 'dict'>, {'Assembly-Data': {'Assembly Method': 'Torrent Suite SARS-CoV-2 Plugin v. 2023', 'Sequencing Technology': 'IonTorrent'}})
Seq('GTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...TGA')


In [ ]:
from Bio import SeqIO
for idx, record in enumerate(SeqIO.parse("/content/data/raw/SARS-CoV-2.fasta", "fasta")):
    if idx < 5:
        print(record.id)
        # remove the id from the description
        # str.partition will put the delimiter as the 2nd element of the list
        describe = [s for idx, s in enumerate(record.description.partition(" ")) if idx > 1][0]
        print(describe)
        print(record.seq)

PP757787.1
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NY-PBRI-64844/2024, complete genome
...
PP757786.1
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NY-PBRI-64848/2024, complete genome
...
PP756578.1
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NY-PRL-240430_81C12/2024 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), nucleocapsid phosphoprotein (N), and ORF10 protein (ORF10) genes, complete cds
...
PP756577.1
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NY-PRL-240430_81C10/2024 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), O

Write the sequences to a FASTA file

In [ ]:
# read the 2nd, 3rd, 4th sequence from the COVID FASTA file
# then write these three sequences to a new FASTA file

seqs = []
for idx, record in enumerate(SeqIO.parse("/content/data/raw/SARS-CoV-2.fasta", "fasta")):
    if idx in [1, 2, 3]:
        seqs.append(record.seq)

In [ ]:
for seq in seqs:
    print(seq)

...


In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

# use the id, name, and description tags to add meta data to the FASTA file

records = []
for idx, record in enumerate(seqs) :
    record = SeqRecord(record, id='seq_'+str(idx), description='covid-19 viral sequence from GenBank')
    records.append(record)

with open("covid_example_seq.fasta", "w") as output_handle:
    SeqIO.write(records, output_handle, "fasta")

In [ ]:
# read our newly written fasta file and print out the content

for record in enumerate(SeqIO.parse("covid_example_seq.fasta", "fasta")):
    print(record)

(0, SeqRecord(seq=Seq('GTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...TAT'), id='seq_0', name='seq_0', description='seq_0 covid-19 viral sequence from GenBank', dbxrefs=[]))
(1, SeqRecord(seq=Seq('CTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCG...GTG'), id='seq_1', name='seq_1', description='seq_1 covid-19 viral sequence from GenBank', dbxrefs=[]))
(2, SeqRecord(seq=Seq('CTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCT...GAT'), id='seq_2', name='seq_2', description='seq_2 covid-19 viral sequence from GenBank', dbxrefs=[]))


In [ ]:
# write raw string as nucleotide sequence into a fasta file
from Bio.Seq import Seq

sequences = ["AAACGTGG", "TGAACCG", "GGTGCA", "CCAATGCG"]
new_sequences = []
i = 1
for seq in sequences:
    #directly using a string as sequence will raise a warning, so convert the string to a sequence object before put it into the SeqRecord
    record = SeqRecord(Seq(seq), id="Seq_"+str(i), description="<custom description>")
    new_sequences.append(record)
    i += 1
with open("example.fasta", "w") as output_handle:
    SeqIO.write(new_sequences, output_handle, "fasta")

In [ ]:
for record in enumerate(SeqIO.parse("example.fasta", "fasta")):
    print(record)

(0, SeqRecord(seq=Seq('AAACGTGG'), id='Seq_1', name='Seq_1', description='Seq_1 <custom description>', dbxrefs=[]))
(1, SeqRecord(seq=Seq('TGAACCG'), id='Seq_2', name='Seq_2', description='Seq_2 <custom description>', dbxrefs=[]))
(2, SeqRecord(seq=Seq('GGTGCA'), id='Seq_3', name='Seq_3', description='Seq_3 <custom description>', dbxrefs=[]))
(3, SeqRecord(seq=Seq('CCAATGCG'), id='Seq_4', name='Seq_4', description='Seq_4 <custom description>', dbxrefs=[]))


## Convert GenBank file to FASTA file
FASTQ files are similar to FASTA but also contain the quality score of the sequence data (only nucleotide sequences). The format contains two additional lines beyond FASTA format.
To acquire FASTQ file, we use the NCBI RSA database

In [ ]:
with open("/content/data/raw/SARS-CoV-2.gbk", "r") as input_handle:
    with open("file.fasta", "w") as output_handle:
        sequences = SeqIO.parse(input_handle, "genbank")
        count = SeqIO.write(sequences, output_handle, "fasta")

print(f"Converted {count} records")

Converted 20 records


# Search literature from Pubmed

The Bio.Entrez API provides us a tool to search the PubMed dataset with Python

In [ ]:
from Bio import Entrez
import pandas as pd
import numpy as np

The search function to return the ID of the papers

In [ ]:
# define a function to search PubMed by a text query
# we limit the max returned hit items to 200, you can extend this number to meet your research purpose.

def search(query):
    Entrez.email = 'your.email@example.com'
    # for demo purpose, we only ask Pubmed to return the top 50 paper IDs
    handle = Entrez.esearch(db='pubmed', sort='relevance', retmax='200', retmode='xml', term=query)
    results = Entrez.read(handle)
    return results

In [ ]:
# search published papers relevant to covid-19

query = 'covid-19 clinical research'

results = search(query)
print(results.keys())
print(f"Total hits: {results['Count']}")
print(results['IdList'])
print(f"Total # of returned paper IDs: {len(results['IdList'])}")

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'QueryTranslation'])
Total hits: 77110
['33400058', '34755408', '33305456', '36995773', '33980687', '35669787', '33629336', '35619009', '33322035', '33190302', '34942102', '33394144', '33975005', '36508742', '32761898', '33930607', '32330303', '34355645', '32383182', '33199136', '33057203', '34910859', '33733663', '33595397', '34353275', '33359141', '33960723', '33090877', '32890414', '33976001', '33622845', '34787548', '34158795', '34131407', '33760236', '34087220', '34119845', '37653342', '34689348', '35215772', '35602124', '34607509', '34287238', '33540323', '36333051', '33931780', '35859397', '33172592', '35995361', '33896332', '33220695', '33744623', '35184764', '33617597', '32703064', '33725432', '37771594', '33593219', '37436038', '33960839', '32856766', '32582617', '37132285', '33689569', '33296409', '32876570', '33439059', '37053243', '34294751', '33034095', '33895475', '33886596', '33662986', '33837673', '3

After we got the paper IDs, we can fetech the details of the citations by the paper IDs

In [ ]:
ids = ','.join(results['IdList'])
ids

'33400058,34755408,33305456,36995773,33980687,35669787,33629336,35619009,33322035,33190302,34942102,33394144,33975005,36508742,32761898,33930607,32330303,34355645,32383182,33199136,33057203,34910859,33733663,33595397,34353275,33359141,33960723,33090877,32890414,33976001,33622845,34787548,34158795,34131407,33760236,34087220,34119845,37653342,34689348,35215772,35602124,34607509,34287238,33540323,36333051,33931780,35859397,33172592,35995361,33896332,33220695,33744623,35184764,33617597,32703064,33725432,37771594,33593219,37436038,33960839,32856766,32582617,37132285,33689569,33296409,32876570,33439059,37053243,34294751,33034095,33895475,33886596,33662986,33837673,36596443,35698094,34794240,33823529,34305894,32779772,33095982,33819290,35173731,33880176,33390829,33730868,35445404,32859676,35441745,35171037,35137006,32598557,33576528,34000256,33951374,38299114,33754328,34852410,35462298,33691216,33428981,36964000,33894979,35141190,34174375,36799012,37074202,32813575,34960660,34223401,36827972,

In [ ]:
# define a function to fetch the details of the hit literature items

def fetch_details(id_list):
  # convert the list of IDs into a long string, each ID separated by ','
  ids = ','.join(id_list)
  Entrez.email = 'email@example.com'
  handle = Entrez.efetch(db='pubmed',retmode='xml',id=ids)
  results = Entrez.read(handle)
  # the return is a complex json structure data structure
  return results

In [ ]:
study_ids = results['IdList']

title_list= []
abstract_list=[]
journal_list = []
language_list =[]
pubdate_year_list = []
pubdate_month_list = []

studies = fetch_details(study_ids)
chunk_size = 10
for chunk_i in range(0, len(study_ids), chunk_size):
    chunk = study_ids[chunk_i:chunk_i + chunk_size]
    papers = fetch_details(chunk)
    for i, paper in enumerate (papers['PubmedArticle']):
        title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])
        try:
            abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
        except:
            abstract_list.append('No Abstract')
        journal_list.append(paper['MedlineCitation']['Article']['Journal']['Title'])
        language_list.append(paper['MedlineCitation']['Article']['Language'][0])
        try:
            pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
        except:
            pubdate_year_list.append('No Data')
        try:
            pubdate_month_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month'])
        except:
            pubdate_month_list.append('No Data')

Put all the data into a Panda Data Frame

In [ ]:
df = pd.DataFrame(list(zip(title_list, abstract_list, journal_list, language_list, pubdate_year_list, pubdate_month_list)),
                  columns=['Title', 'Abstract', 'Journal', 'Language', 'Year','Month'],
                  )


In [ ]:
df.head(10)

,Title,Abstract,Journal,Language,Year,Month
0,Recent Developments on Therapeutic and Diagnos...,The ongoing pandemic of coronavirus disease 20...,The AAPS journal,eng,2021,Jan
1,Covid-19 vaccines and variants of concern: A r...,Since the outbreak of coronavirus disease 2019...,Reviews in medical virology,eng,2022,Jul
2,"COVID-19: Virology, biology and novel laborato...","At the end of December 2019, a novel coronavir...",The journal of gene medicine,eng,2021,Feb
3,Tracking the COVID-19 vaccines: The global lan...,COVID-19: Coronavirus disease 2019; SARS-CoV-2...,Human vaccines & immunotherapeutics,eng,2023,Dec
4,Tools and Techniques for Severe Acute Respirat...,The coronavirus disease 2019 (COVID-19) pandem...,Clinical microbiology reviews,eng,2021,Jun
5,Role of COVID-19 Vaccines in SARS-CoV-2 Variants.,"Coronavirus disease 2019 (COVID-19), caused by...",Frontiers in immunology,eng,2022,No Data
6,"COVID-19 vaccines: comparison of biological, p...","The ""Severe Acute Respiratory Syndrome Coronav...",European review for medical and pharmacologica...,eng,2021,Feb
7,COVID-19 diagnostic methods in developing coun...,COVID-19 has become one of the few leading cau...,Environmental science and pollution research i...,eng,2022,Jul
8,Comparison of Rapid Antigen Tests for COVID-19.,Reverse transcription-quantitative PCR (RT-qPC...,Viruses,eng,2020,Dec
9,Coronavirus disease 2019 (COVID-19): An overvi...,SARS-CoV-2 is a novel human coronavirus respon...,Scandinavian journal of immunology,eng,2021,Apr


In [ ]:
df['Month'].replace('Jan', '01', inplace=True)
df['Month'].replace('Feb', '02', inplace=True)
df['Month'].replace('Mar', '03', inplace=True)
df['Month'].replace('Apr', '04', inplace=True)
df['Month'].replace('May', '05', inplace=True)
df['Month'].replace('Jun', '06', inplace=True)
df['Month'].replace('Jul', '07', inplace=True)
df['Month'].replace('Aug', '08', inplace=True)
df['Month'].replace('Sep', '09', inplace=True)
df['Month'].replace('Oct', '10', inplace=True)
df['Month'].replace('Nov', '11', inplace=True)
df['Month'].replace('Dec', '12', inplace=True)
# use "np.nan" to replace "No Data"
df['Month'].replace('No Data', np.nan, inplace=True)

In [ ]:
print(len(df))

200


With this structure data frame, we can do some NLP research given some topics.